In [1]:
import pyspark
from pyspark.sql import SparkSession

bucket = "labe"

spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
        .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2")\
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
        .config("spark.hadoop.fs.s3a.access.key", "minio") \
        .config("spark.hadoop.fs.s3a.secret.key", "SU2orange!") \
        .config("spark.hadoop.fs.s3a.fast.upload", True) \
        .config("spark.hadoop.fs.s3a.path.style.access", True) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR") # Keeps the noise down!!!

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-53104aa5-2b13-47fb-84ad-b28b851154aa;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-aws;3.1.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.1.2/hadoop-aws-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.1.2!hadoop-aws.jar (150ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.271/aws-java-sdk-bundle-1.11.271.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.11.271!aws-java-sdk-bundle.jar (63722ms)
:: resolution report :: resolve 1710ms :: artifacts dl 63876ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.271 from central in [default]
	org.apache.hadoop#hadoop-aws;3.1.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------------------------------------------------

In [4]:
weather = spark.read\
    .option("header",True).option("inferSchema",True)\
    .csv("s3a://labe/syracuse-ny.csv")

weather.printSchema()

root
 |-- EST: string (nullable = true)
 |-- Max TemperatureF: integer (nullable = true)
 |-- Mean TemperatureF: integer (nullable = true)
 |-- Min TemperatureF: integer (nullable = true)
 |-- Max Dew PointF: integer (nullable = true)
 |-- MeanDew PointF: integer (nullable = true)
 |-- Min DewpointF: integer (nullable = true)
 |-- Max Humidity: integer (nullable = true)
 |-- Mean Humidity: integer (nullable = true)
 |-- Min Humidity: integer (nullable = true)
 |-- Max Sea Level PressureIn: double (nullable = true)
 |-- Mean Sea Level PressureIn: double (nullable = true)
 |-- Min Sea Level PressureIn: double (nullable = true)
 |-- Max VisibilityMiles: integer (nullable = true)
 |-- Mean VisibilityMiles: integer (nullable = true)
 |-- Min VisibilityMiles: integer (nullable = true)
 |-- Max Wind SpeedMPH: integer (nullable = true)
 |-- Mean Wind SpeedMPH: integer (nullable = true)
 |-- Max Gust SpeedMPH: integer (nullable = true)
 |-- PrecipitationIn: string (nullable = true)
 |-- CloudCo

In [3]:
spark.read\
    .option("header",True).option("inferSchema",True)\
    .csv("s3a://labe/syracuse-ny.csv")\
    .createOrReplaceTempView("weather")

spark.sql("select * from weather").printSchema()

root
 |-- EST: string (nullable = true)
 |-- Max TemperatureF: integer (nullable = true)
 |-- Mean TemperatureF: integer (nullable = true)
 |-- Min TemperatureF: integer (nullable = true)
 |-- Max Dew PointF: integer (nullable = true)
 |-- MeanDew PointF: integer (nullable = true)
 |-- Min DewpointF: integer (nullable = true)
 |-- Max Humidity: integer (nullable = true)
 |-- Mean Humidity: integer (nullable = true)
 |-- Min Humidity: integer (nullable = true)
 |-- Max Sea Level PressureIn: double (nullable = true)
 |-- Mean Sea Level PressureIn: double (nullable = true)
 |-- Min Sea Level PressureIn: double (nullable = true)
 |-- Max VisibilityMiles: integer (nullable = true)
 |-- Mean VisibilityMiles: integer (nullable = true)
 |-- Min VisibilityMiles: integer (nullable = true)
 |-- Max Wind SpeedMPH: integer (nullable = true)
 |-- Mean Wind SpeedMPH: integer (nullable = true)
 |-- Max Gust SpeedMPH: integer (nullable = true)
 |-- PrecipitationIn: string (nullable = true)
 |-- CloudCo

In [6]:
query = '''
with source as  (
    select 
        cast(split(EST,'-')[0] as int) as year,
        cast(split(EST,'-')[1] as int) as month,
        `Min TemperatureF` as mintemp,
        `Max TemperatureF` as maxtemp
    from weather
)
select 
    year, month, avg(mintemp) as avgmin, avg(maxtemp) as avgmax
    from source 
    group by year, month
    order by year, month

'''
spark.sql(query).explain()

== Physical Plan ==
*(3) Sort [year#191 ASC NULLS FIRST, month#192 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(year#191 ASC NULLS FIRST, month#192 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#97]
   +- *(2) HashAggregate(keys=[year#191, month#192], functions=[avg(cast(mintemp#193 as bigint)), avg(cast(maxtemp#194 as bigint))])
      +- Exchange hashpartitioning(year#191, month#192, 200), ENSURE_REQUIREMENTS, [id=#93]
         +- *(1) HashAggregate(keys=[year#191, month#192], functions=[partial_avg(cast(mintemp#193 as bigint)), partial_avg(cast(maxtemp#194 as bigint))])
            +- *(1) Project [cast(split(EST#16, -, -1)[0] as int) AS year#191, cast(split(EST#16, -, -1)[1] as int) AS month#192, Min TemperatureF#19 AS mintemp#193, Max TemperatureF#17 AS maxtemp#194]
               +- FileScan csv [EST#16,Max TemperatureF#17,Min TemperatureF#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[s3a://labe/syracuse-ny.csv], PartitionFilters: [], P

In [9]:
query = '''
with source as  (
    select 
        cast(split(EST,'-')[0] as int) as year,
        cast(split(EST,'-')[1] as int) as month,
        `Min TemperatureF` as mintemp,
        `Max TemperatureF` as maxtemp
    from weather
)
select 
    year, month, avg(mintemp) as avgmin, avg(maxtemp) as avgmax
    from source 
    group by year, month
    order by year, month

'''
spark.sql(query).createOrReplaceTempView("monthly_syracuse_weather_averages")

spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |monthly_syracuse_...|       true|
|        |             weather|       true|
+--------+--------------------+-----------+



In [11]:
month = input("Enter Month: ")
spark.sql(f"select * from monthly_syracuse_weather_averages where month = {month}").show()

Enter Month:  7


+----+-----+------------------+-----------------+
|year|month|            avgmin|           avgmax|
+----+-----+------------------+-----------------+
|1997|    7| 59.87096774193548|80.19354838709677|
|1998|    7| 61.29032258064516|79.03225806451613|
|1999|    7|  64.3225806451613|85.74193548387096|
|2000|    7|57.774193548387096|76.51612903225806|
|2001|    7|              59.0|79.87096774193549|
|2002|    7| 63.61290322580645|84.16129032258064|
|2003|    7|61.483870967741936|81.29032258064517|
|2004|    7|60.903225806451616|78.16129032258064|
|2005|    7| 64.19354838709677|85.12903225806451|
|2006|    7|              65.0|83.12903225806451|
|2007|    7|59.483870967741936|80.19354838709677|
|2008|    7| 61.41935483870968|81.19354838709677|
|2009|    7|58.935483870967744|77.45161290322581|
|2010|    7| 64.38709677419355|84.64516129032258|
|2011|    7| 64.64516129032258|86.93548387096774|
|2012|    7| 64.54838709677419|87.90322580645162|
|2013|    7| 65.06451612903226|83.83870967741936|


In [13]:
spark.sql(f"select max(year),max(month) from monthly_syracuse_weather_averages").show()

+---------+----------+
|max(year)|max(month)|
+---------+----------+
|     2015|        12|
+---------+----------+



In [12]:
from IPython.display import display, HTML
from ipywidgets import interact_manual

display(HTML("<H1>Pick a Month</h1>") )
@interact_manual(Month=(1,12), Year=[2020, 2021, 2022])
def onclick(Month,Year):
    display(HTML(f"You selected: <b>{Month}</b> <em>{Year}</em>"))
    
# type = tuple = slider of int/floats
# type = string = textbox
# type = list = dropdown
# type = boolean = checkbox

interactive(children=(IntSlider(value=6, description='Month', max=12, min=1), Dropdown(description='Year', opt…

In [16]:
@interact_manual(name="", is_grad=False, is_su=False, gpa=(0.0,4.0) )
def onclick(name, is_grad, is_su, gpa):
    print("do your analysis")
    print(name, is_grad, is_su, gpa)

interactive(children=(Text(value='', description='name'), Checkbox(value=False, description='is_grad'), Checkb…

In [14]:
from IPython.display import display, HTML
from ipywidgets import interact_manual
import matplotlib.pyplot as plt

display(HTML("<H1>Syracuse Weather</h1>") )
@interact_manual(Month=(1,12))
def doit(Month):
    df = spark.sql(f"select * from monthly_syracuse_weather_averages where month = {Month}").toPandas()
    display(df)
    df.set_index("year")
    plt.figure(figsize=(15,10))
    plt.scatter(df["year"],y=df["avgmin"], label='monthly avg min', marker='v')
    plt.scatter(df["year"],y=df["avgmax"], label='monthly avg max', marker='^')
    plt.legend(title=f"Temps for Month {Month}")
    plt.show()
    

interactive(children=(IntSlider(value=6, description='Month', max=12, min=1), Button(description='Run Interact…

In [17]:
reddit = spark.read.json("s3a://labe/reddit.json")

In [21]:
reddit.select("data.children").show(5)

+--------------------+
|            children|
+--------------------+
|[{{null, false, h...|
+--------------------+



In [22]:
from pyspark.sql.functions import explode,split

In [34]:
reddit = spark.read.json("s3a://labe/reddit.json")

x =reddit.select( explode( "data.children") )

y = x.select("col.data").select("data.id","data.name","data.title","data.author","data.score","data.url")

y.createOrReplaceTempView("reddit")

spark.sql("select * from reddit").write.mode("overwrite").csv("s3a://labe/reddit.csv",header=True)
